In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from WattPredictor.utils.helpers import *
from WattPredictor.utils.exception import *
from WattPredictor.constants import *
from WattPredictor import logger

In [3]:
@dataclass
class ModelEvaluationConfig:
    model_path: Path
    x_transform: Path
    y_transform: Path
    metrics_path: Path

@dataclass(frozen=True)
class FeatureStoreConfig:
    hopsworks_project_name: str
    hopsworks_api_key: str

In [4]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_PATH,
                 params_filepath = PARAMS_PATH,
                 schema_filepath = SCHEMA_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        model_evaluation_config =  ModelEvaluationConfig(
            model_path=Path(config.model_path),
            x_transform=Path(config.X_transform),
            y_transform=Path(config.y_transform),
            metrics_path=Path(config.metrics_path)
        )

        return model_evaluation_config
    
    def get_feature_store_config(self) -> FeatureStoreConfig:

        config = self.config.feature_store

        feature_store_config = FeatureStoreConfig(
                hopsworks_project_name=config.hopsworks_project_name,
                hopsworks_api_key=os.environ['hopsworks_api_key'],
        )

        return feature_store_config

In [5]:
import hopsworks
import pandas as pd
import sys
import os
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger

class FeatureStore:
    def __init__(self, config):
        try:
            self.config = config
            self.connect()
        except Exception as e:
            raise CustomException(e, sys)

    def connect(self):
        try:
            self.project = hopsworks.login(
                project=self.config.hopsworks_project_name,
                api_key_value=self.config.hopsworks_api_key
            )
            self.feature_store = self.project.get_feature_store()
            self.dataset_api = self.project.get_dataset_api()
            logger.info(f"Connected to Hopsworks Feature Store: {self.config.hopsworks_project_name}")
        except Exception as e:
            raise CustomException(e, sys)

    def create_feature_group(self, name, df, primary_key, event_time, description):
        try:
            try:
                fg = self.feature_store.get_feature_group(name=name, version=1)
                logger.info(f"Feature Group '{name}' already exists. Inserting data instead.")
                fg.insert(df)
            except:
                logger.info(f"Feature Group '{name}' does not exist. Creating new one.")
                fg = self.feature_store.get_or_create_feature_group(
                    name=name,
                    version=1,
                    primary_key=primary_key,
                    event_time=event_time,
                    description=description,
                    online_enabled=False
                )
                fg.save(df)

            logger.info(f"Feature Group '{name}' created/updated successfully")

        except Exception as e:
            raise CustomException(e, sys)

    def create_feature_view(self, name: str, feature_group_name: str, features: list):
        try:
            fg = self.feature_store.get_feature_group(name=feature_group_name, version=1)
            fv = self.feature_store.get_or_create_feature_view(
                name=name,
                version=1,
                query=fg.select(features),
                description=f"Feature View for {name}"
            )
            logger.info(f"Feature View '{name}' created successfully")
        except Exception as e:
            raise CustomException(e, sys)

    def upload_file_safely(self, local_path: str, target_name: str):
        """
        Upload file to Hopsworks dataset storage.
        If it already exists, it will be overwritten.
        """
        try:
            self.dataset_api.upload(
                local_path,
                f"Resources/wattpredictor_artifacts/{target_name}",
                overwrite=True 
            )
            logger.info(f"Uploaded file to Feature Store: {target_name}")
        except Exception as e:
            raise CustomException(e, sys)
        
    def download_file(self, remote_name: str, local_path: str = None):
        """
        Download a file from Hopsworks dataset storage.

        Args:
            remote_name: filename in Hopsworks (inside wattpredictor_artifacts)
            local_path: optional local path to save the file. If None, saves in current directory.
        """
        try:
            target_path = f"Resources/wattpredictor_artifacts/{remote_name}"
            if local_path is None:
                local_path = remote_name

            self.dataset_api.download(
                target_path,
                local_path=local_path,
                overwrite=True
            )
            logger.info(f"Downloaded file from Feature Store: {remote_name} to {local_path}")

        except Exception as e:
            raise CustomException(e, sys)


    def delete_file(self, target_name: str):
        """
        Delete file from Hopsworks dataset storage.
        Only use this if you want to clean up files manually.
        """
        try:
            full_path = f"Resources/wattpredictor_artifacts/{target_name}"
            self.dataset_api.delete(full_path)
            logger.warning(f"Deleted file from Feature Store: {target_name}")
        except Exception as e:
            logger.warning(f"File not found or already deleted: {target_name}")
            # Not raising exception here to allow safe cleanup

    def get_training_data(self, feature_view_name: str):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            X, y = fv.training_data()
            logger.info(f"Retrieved training data from Feature View '{feature_view_name}'")
            return X, y
        except Exception as e:
            raise CustomException(e, sys)

In [6]:
import os
import sys
import json
import joblib
import mlflow
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

from WattPredictor.utils.helpers import create_directories, save_json
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig, feature_store_config):
        self.config = config
        self.feature_store_config = feature_store_config
        self.feature_store = FeatureStore(feature_store_config)

        mlflow.set_tracking_uri("file:./mlruns")
        mlflow.set_experiment("Electricity Demand Prediction")
        logger.info("MLflow tracking setup complete.")

    def download_inputs(self):
        try:

            self.feature_store.dataset_api.download("Resources/wattpredictor_artifacts/model.joblib/model.joblib", overwrite=True)
            self.feature_store.dataset_api.download("Resources/wattpredictor_artifacts/test_x.parquet/test_x.parquet", overwrite=True)
            self.feature_store.dataset_api.download("Resources/wattpredictor_artifacts/test_y.parquet/test_y.parquet", overwrite=True)

            test_x = pd.read_parquet(self.config.x_transform)
            test_y = pd.read_parquet(self.config.y_transform)
            
            test_x = test_x.values
            test_y = test_y.squeeze().values 
            model = joblib.load(self.config.model_path)

            logger.info(f'shape of train_x:{test_x.shape}, train_y:{test_y.shape}')

            return test_x,test_y, model

        except Exception as e:
            raise CustomException(e, sys)

    def evaluate(self):
        try:
            test_x,test_y, model = self.download_inputs()


            # Predict
            preds = model.predict(test_x)

            # Metrics
            metrics = {
                "mse": mean_squared_error(test_y, preds),
                "mae": mean_absolute_error(test_y, preds),
                "rmse": np.sqrt(mean_squared_error(test_y, preds)),
                "mape": np.mean(np.abs((test_y - preds) / test_y)) * 100 if np.any(test_y != 0) else np.inf,
                "r2_score": r2_score(test_y, preds),
                "adjusted_r2": 1 - (1 - r2_score(test_y, preds)) * (len(test_y) - 1) / (len(test_y) - test_x.shape[1] - 1)
            }

            create_directories([Path(self.config.metrics_path).parent])
            save_json(Path(self.config.metrics_path), metrics)

            logger.info(f"📊 Evaluation Metrics: {metrics}")

            # Log to MLflow
            with mlflow.start_run(run_name="Model Evaluation"):
                mlflow.log_metrics({k: float(v) for k, v in metrics.items()})
                mlflow.set_tag("stage", "evaluation")
                mlflow.log_artifact(self.config.metrics_path)
                mlflow.log_artifact(self.config.model_path)

            logger.info("✅ Model evaluation complete and metrics logged.")
            return metrics

        except Exception as e:
            raise CustomException(e, sys)

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    feature_store_config = config.get_feature_store_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config, feature_store_config=feature_store_config)
    model_evaluation.evaluate()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-07-11 15:05:53,024: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-11 15:05:53,028: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-11 15:05:53,031: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-11 15:05:53,032: INFO: helpers: created directory at: artifacts]
[2025-07-11 15:05:53,034: INFO: external: Initializing external client]
[2025-07-11 15:05:53,036: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-11 15:05:55,914: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1237149
[2025-07-11 15:05:58,478: INFO: 468788050: Connected to Hopsworks Feature Store: JavithNaseem]
[2025-07-11 15:05:58,485: WARNING: file_store: Malformed experiment '141997247217478997'. Detailed error Yaml file '.\mlruns\141997247217478997\meta.yaml' does not exist.]
Traceback (most recent call last):
  File "f:\Program Files\anac

Downloading: 0.000%|          | 0/1216576 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/5889601 elapsed<00:00 remaining<?

[2025-07-11 15:08:23,798: WARNING: connectionpool: Retrying (Retry(total=0, connect=None, read=False, redirect=None, status=None)) after connection broken by 'MustRedialError('Remote peer just closed our connection, probably for not answering to unsolicited packet. (None)')': /hopsworks-api/api/project/1237149/dataset/Resources%2Fwattpredictor_artifacts%2Ftest_y.parquet%2Ftest_y.parquet]


Downloading: 0.000%|          | 0/9758 elapsed<00:00 remaining<?

[2025-07-11 15:08:26,080: INFO: 1575915400: shape of train_x:(2112, 674), train_y:(2112,)]
[2025-07-11 15:08:26,085: WARNING: warnings: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
]
[2025-07-11 15:08:31,912: INFO: helpers: created directory at: artifacts\model_evaluation]
[2025-07-11 15:08:31,915: INFO: helpers: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-11 15:08:31,916: INFO: 1575915400: 📊 Evaluation Metrics: {'mse': 8741.435722876593, 'mae': 51.382026308241464, 'rmse': 93.49564547547973, 'mape': 4.428398874064529, 'r2_score': 0.9944144248200268, 'adjusted_r2': 0.9917946073730527}]
[2025-07-11 15:08:32,314: INFO: 1575915400: ✅ Model evaluation complete and metrics logged.]
